# Feature Engineering Workshop for Analytics Vidhya

This wokshop will walk through incrementally building feature from transactional data for a customer behavior use case. The data is synthetic and was created to support a churn use case.

This notebook can be found [here](https://github.com/rasgointelligence/feature-engineering-tutorials/tree/main/workshops/analytics-vidhya/workshop_2022_02.ipynb)

## Packages

The documentation for each packaged used in this tutorial is linked below:
* [numpy](https://numpy.org/doc/stable/)
* [pandas](https://pandas.pydata.org/docs/)
* [scikit-learn](https://scikit-learn.org/stable/)
    * [sklearn.preprocessing](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing)
* [category_encoders](https://contrib.scikit-learn.org/category_encoders/)

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import category_encoders as ce

## Get customer and transaction data

Grab customer and transcation data from GitHub repo and 

In [2]:
customer_url = "https://raw.githubusercontent.com/engelAnalytics/feature-engineering/main/data/churn/customer.csv"
transactions_url = "https://raw.githubusercontent.com/engelAnalytics/feature-engineering/main/data/churn/transactions.csv"

customer = pd.read_csv(customer_url, parse_dates=['Birthdate', 'Churn_date'])
transactions = pd.read_csv(transactions_url, parse_dates=['Transaction_Date'])
customer.columns

Index(['CustomerID', 'Gender', 'Birthdate', 'City', 'State', 'ZIP',
       'Customer_since', 'Country', 'Phone', 'Churn_date'],
      dtype='object')

In [3]:
transactions.columns

Index(['CustomerID', 'TransactionID', 'Transaction_Date', 'Product',
       'Product_Count', 'Cost'],
      dtype='object')

## Create moving aggregates

Moving, or rolling, aggregates are a very powerful way to summarize transactional data. pandas makes this easy through the use of the `rolling` function. By default, `rolling` simply rolls up the prior *n* number of transactions. This can be useful. The behavior of a customer over the prior 5 or 10 transactions can be predictive and is worth exploring. In this case, as there can be a wide range of purchase frequency, moving windows based on dates can be more useful.  `rolling` supports this as well either through an additional parameter or by setting a datatime field as an index.

In addition to a moving average over the transaction date, these need to be done at the customer level, so the data is grouped by CustomerID with `groupby`

In [4]:
rolling_window_180 = transactions.sort_values(by=['CustomerID', 'Transaction_Date']).set_index('Transaction_Date').groupby('CustomerID').rolling('180D', closed='both')

With this rolling group, different aggregations can be performed. For ease of readability, this is being done one variable at a time, but `agg` could be called to perform all of these simultaneously.

In [5]:
agg_df = pd.DataFrame()
agg_df['Cost_min_180D'] = rolling_window_180["Cost"].min()
agg_df['Cost_max_180D'] = rolling_window_180["Cost"].max()
agg_df['Cost_mean_180D'] = rolling_window_180["Cost"].mean()
agg_df['Cost_std_180D'] = rolling_window_180["Cost"].std()
agg_df['Cost_sum_180D'] = rolling_window_180["Cost"].sum()

Rolling statistics for the total purchase amount (*Cost* in the data) has been created. The same approach could be used to generate rolling statistics for other quantites (e.g. *Product_Count*).

Beyond these statistics, knowing how many transactions have occured in the time window is helpful. This can easily be calculated from the rolling window we have already created.

In [6]:
agg_df['Purchase_count_180D'] = rolling_window_180["Cost"].count()

Merging these back into the original transaction data allows us to compare the aggregations to the raw data.  We are not ready to merge all the data together, but the calculations can be checked.

In [7]:
transactions.merge(agg_df.reset_index(), on=['CustomerID', 'Transaction_Date'], how='left').head(20)

,CustomerID,TransactionID,Transaction_Date,Product,Product_Count,Cost,Cost_min_180D,Cost_max_180D,Cost_mean_180D,Cost_std_180D,Cost_sum_180D,Purchase_count_180D
0,180599,118059920170114,2017-01-14,B,2,300,300.0,300.0,300.000000,NaN,300.0,1.0
1,180599,218059920170218,2017-02-18,C,3,225,225.0,300.0,262.500000,53.033009,525.0,2.0
2,180599,318059920170501,2017-05-01,C,2,150,150.0,300.0,225.000000,75.000000,675.0,3.0
3,180599,418059920170718,2017-07-18,A,2,200,150.0,225.0,191.666667,38.188131,575.0,3.0
4,180599,518059920171016,2017-10-16,B,2,300,150.0,300.0,216.666667,76.376262,650.0,3.0
5,180599,618059920171115,2017-11-15,A,2,200,200.0,300.0,233.333333,57.735027,700.0,3.0
6,180599,718059920180202,2018-02-02,B,3,450,200.0,450.0,316.666667,125.830574,950.0,3.0
7,180599,818059920180326,2018-03-26,B,2,300,200.0,450.0,312.500000,103.077641,1250.0,4.0
8,180599,918059920180508,2018-05-08,C,2,150,150.0,450.0,275.000000,132.287566,1100.0,4.0
9,180599,1018059920180802,2018-08-02,A,2,200,150.0,300.0,216.666667,76.376262,650.0,3.0


### Product level aggregations

Feature engineering for categorical variables tends to focus on just a few techniques to convert the values to numbers. With transactional data, these categories can be converted to numerical by aggregating all the prior behavior for these categories much like was just done at the customer level.  Due to some issues with the way `rolling` behaves, likely related to duplicate indicies, we will split the data at the group level and calculate each separately.

There is one complication as there can be multiple transactions for a product on a given date. To avoid issues, we will only include transactions from days prior to this one. `rolling` supports this with the parameter *closed*. By setting it to *left*, the most recent date is excluded.

Finally, we drop the duplicate keeping only the first record that is based on calculations from prior days only.

In [8]:
prod_df = pd.DataFrame()
for prod in sorted(transactions['Product'].unique()):
    product_window_180 = transactions[transactions.Product == prod].sort_values(by=['Transaction_Date']).set_index('Transaction_Date').rolling('180D', closed='left')
    
    tmpdf = pd.DataFrame()
    tmpdf['Product_Cost_mean_180D'] = product_window_180["Cost"].mean()
    tmpdf['Product_Cost_std_180D'] = product_window_180["Cost"].std()
    tmpdf['Product_count_180D'] = product_window_180["Product_Count"].count()
    tmpdf['Product'] = prod
    
    tmpdf = tmpdf.reset_index().sort_values(by=['Transaction_Date', 'Product_count_180D']).drop_duplicates(subset=['Transaction_Date'], keep='first')
    prod_df = pd.concat([prod_df, tmpdf])
    
prod_df.head()

,Transaction_Date,Product_Cost_mean_180D,Product_Cost_std_180D,Product_count_180D,Product
0,2017-01-02,NaN,NaN,NaN,A
1,2017-01-04,700.000000,NaN,1.0,A
3,2017-01-05,400.000000,264.575131,3.0,A
9,2017-01-06,477.777778,253.859104,9.0,A
18,2017-01-07,494.444444,233.823016,18.0,A


The merge is very similar to the one for customers.

In [9]:
transactions.merge(prod_df, on=['Product', 'Transaction_Date'], how='left').sort_values(by=['Product', 'Transaction_Date']).head(20)

,CustomerID,TransactionID,Transaction_Date,Product,Product_Count,Cost,Product_Cost_mean_180D,Product_Cost_std_180D,Product_count_180D
20569,181906,118190620170102,2017-01-02,A,7,700,NaN,NaN,NaN
20160,177614,117761420170104,2017-01-04,A,2,200,700.000000,NaN,1.0
35100,177400,117740020170104,2017-01-04,A,3,300,700.000000,NaN,1.0
638,180289,118028920170105,2017-01-05,A,4,400,400.000000,264.575131,3.0
1975,181757,118175720170105,2017-01-05,A,6,600,400.000000,264.575131,3.0
11389,176827,117682720170105,2017-01-05,A,10,1000,400.000000,264.575131,3.0
15612,184872,118487220170105,2017-01-05,A,3,300,400.000000,264.575131,3.0
29398,182989,118298920170105,2017-01-05,A,3,300,400.000000,264.575131,3.0
37518,182841,118284120170105,2017-01-05,A,5,500,400.000000,264.575131,3.0
4639,181375,118137520170106,2017-01-06,A,3,300,477.777778,253.859104,9.0


### State level aggregations

This same approach to the categorical variable in the transactions can also be used for customer level data. For example, we can replace the state with information about how people within the state behave.  In this case, before rolling up by transaction date and grouping by state, the transaction data needs to be merged with the customer data to add the state to the data.

First we need transactions merged with the customer data

In [10]:
full_transactions = transactions.merge(customer, on=['CustomerID'], how='left')

Again the moving average, standard deviation and count are created.

In [11]:
state_df = pd.DataFrame()
for state in sorted(full_transactions['State'].unique()):
    state_window_180 = full_transactions[full_transactions.State == state].sort_values(by=['Transaction_Date']).set_index('Transaction_Date').rolling('180D', closed='left')
    
    tmpdf = pd.DataFrame()
    tmpdf['State_Cost_mean_180D'] = state_window_180["Cost"].mean()
    tmpdf['State_Cost_std_180D'] = state_window_180["Cost"].std()
    tmpdf['State_count_180D'] = state_window_180["Product_Count"].count()
    tmpdf['State'] = state
    
    tmpdf = tmpdf.reset_index().sort_values(by=['Transaction_Date', 'State_count_180D']).drop_duplicates(subset=['Transaction_Date'], keep='first')
    state_df = pd.concat([state_df, tmpdf])
    
state_df.head()

,Transaction_Date,State_Cost_mean_180D,State_Cost_std_180D,State_count_180D,State
0,2017-01-16,NaN,NaN,NaN,AK
1,2017-01-20,300.000000,NaN,1.0,AK
2,2017-01-25,300.000000,0.000000,2.0,AK
3,2017-01-28,283.333333,28.867513,3.0,AK
4,2017-01-30,275.000000,28.867513,4.0,AK


## Merge data

We can now merge the customer, product and state level data together. First merge the transaction and customer data together.

In [12]:
transdf = transactions.merge(customer, on=['CustomerID'], how='left')

Next merge the customer transaction history over time to the transaction data.

In [13]:
transdf = transdf.merge(agg_df.reset_index(), on=['CustomerID', 'Transaction_Date'], how='left')

The product aggregations can be merged into this.

In [14]:
transdf = transdf.merge(prod_df, on=['Product', 'Transaction_Date'], how='left')

Finally. all of this can be merged with the state level aggregations.

In [15]:
transdf = transdf.merge(state_df, on=['State', 'Transaction_Date'], how='left')

transdf.head(20)

,CustomerID,TransactionID,Transaction_Date,Product,Product_Count,Cost,Gender,Birthdate,City,State,...,Cost_mean_180D,Cost_std_180D,Cost_sum_180D,Purchase_count_180D,Product_Cost_mean_180D,Product_Cost_std_180D,Product_count_180D,State_Cost_mean_180D,State_Cost_std_180D,State_count_180D
0,180599,118059920170114,2017-01-14,B,2,300,F,1957-07-29,Bangor,WI,...,300.000000,NaN,300.0,1.0,639.873418,304.494089,79.0,431.666667,285.898750,15.0
1,180599,218059920170218,2017-02-18,C,3,225,F,1957-07-29,Bangor,WI,...,262.500000,53.033009,525.0,2.0,286.950791,146.789196,569.0,376.265823,305.947947,79.0
2,180599,318059920170501,2017-05-01,C,2,150,F,1957-07-29,Bangor,WI,...,225.000000,75.000000,675.0,3.0,286.428571,147.682921,1470.0,383.946078,281.319449,204.0
3,180599,418059920170718,2017-07-18,A,2,200,F,1957-07-29,Bangor,WI,...,191.666667,38.188131,575.0,3.0,377.496038,190.749788,1893.0,391.666667,285.654268,306.0
4,180599,518059920171016,2017-10-16,B,2,300,F,1957-07-29,Bangor,WI,...,216.666667,76.376262,650.0,3.0,552.613828,295.761534,1779.0,383.781362,279.691735,279.0
5,180599,618059920171115,2017-11-15,A,2,200,F,1957-07-29,Bangor,WI,...,233.333333,57.735027,700.0,3.0,356.127886,185.969785,1689.0,377.851711,279.391454,263.0
6,180599,718059920180202,2018-02-02,B,3,450,F,1957-07-29,Bangor,WI,...,316.666667,125.830574,950.0,3.0,513.785885,268.113671,1672.0,342.564655,270.893575,232.0
7,180599,818059920180326,2018-03-26,B,2,300,F,1957-07-29,Bangor,WI,...,312.500000,103.077641,1250.0,4.0,515.185636,271.003929,1643.0,333.139535,260.499214,215.0
8,180599,918059920180508,2018-05-08,C,2,150,F,1957-07-29,Bangor,WI,...,275.000000,132.287566,1100.0,4.0,259.330075,139.695428,1739.0,355.721393,280.075172,201.0
9,180599,1018059920180802,2018-08-02,A,2,200,F,1957-07-29,Bangor,WI,...,216.666667,76.376262,650.0,3.0,352.954210,184.132538,1354.0,359.375000,269.987382,184.0


## Ratios, differences and distances

Many machine learning algorithms have difficulty learning ratios between features. Generating meaningful ratios during feature engineering can improve model performance. An obvious ratio to try is the current purchase amount divided by the average purchase amount over the prior 180 days.

In [16]:
transdf['Cost_ratio'] = transdf['Cost'] / transdf['Cost_mean_180D']

Another interesting ratio could be the current purchase amount divided by the total purchase amount over the prior 180 days.

Tree-based algorithms struggle identifying the signal from the difference between two features. Subtracting the average amount over the prior 180 days from the current purchase could be useful. However, we can take it a step farther.  If we divide that difference by the standard deviation, we have a measure of the distance between the current purchase and the average in terms of standard deviation. This capture how unusual this transaction is for the customer.

In [17]:
transdf['Cost_distance'] = (transdf['Cost'] - transdf['Cost_mean_180D']) / transdf['Cost_std_180D']

In this case, the direction of the difference (bigger or smaller) is captured. Taking the absolute value will capture just how unusual the current transaction is.

In [18]:
transdf['Cost_distance_mag'] = np.abs(transdf['Cost_distance'])

The same ratio and distance can be calculated for the product mean and standard deviation.

In [19]:
transdf['Cost_ratio_product'] = transdf['Cost'] / transdf['Product_Cost_mean_180D']
transdf['Cost_distance_product'] = (transdf['Cost'] - transdf['Product_Cost_mean_180D']) / transdf['Product_Cost_std_180D']
transdf['Cost_distance_product_mag'] = np.abs(transdf['Cost_distance_product'])

And for the state mean and standard deviation.

In [20]:
transdf['Cost_ratio_state'] = transdf['Cost'] / transdf['State_Cost_mean_180D']
transdf['Cost_distance_state'] = (transdf['Cost'] - transdf['State_Cost_mean_180D']) / transdf['State_Cost_std_180D']
transdf['Cost_distance_state_mag'] = np.abs(transdf['Cost_distance_state'])

In [21]:
transdf.head(20)

,CustomerID,TransactionID,Transaction_Date,Product,Product_Count,Cost,Gender,Birthdate,City,State,...,State_count_180D,Cost_ratio,Cost_distance,Cost_distance_mag,Cost_ratio_product,Cost_distance_product,Cost_distance_product_mag,Cost_ratio_state,Cost_distance_state,Cost_distance_state_mag
0,180599,118059920170114,2017-01-14,B,2,300,F,1957-07-29,Bangor,WI,...,15.0,1.000000,NaN,NaN,0.468843,-1.116191,1.116191,0.694981,-0.460536,0.460536
1,180599,218059920170218,2017-02-18,C,3,225,F,1957-07-29,Bangor,WI,...,79.0,0.857143,-0.707107,0.707107,0.784107,-0.422039,0.422039,0.597981,-0.494417,0.494417
2,180599,318059920170501,2017-05-01,C,2,150,F,1957-07-29,Bangor,WI,...,204.0,0.666667,-1.000000,1.000000,0.523691,-0.923794,0.923794,0.390680,-0.831603,0.831603
3,180599,418059920170718,2017-07-18,A,2,200,F,1957-07-29,Bangor,WI,...,306.0,1.043478,0.218218,0.218218,0.529807,-0.930518,0.930518,0.510638,-0.670974,0.670974
4,180599,518059920171016,2017-10-16,B,2,300,F,1957-07-29,Bangor,WI,...,279.0,1.384615,1.091089,1.091089,0.542875,-0.854113,0.854113,0.781695,-0.299549,0.299549
5,180599,618059920171115,2017-11-15,A,2,200,F,1957-07-29,Bangor,WI,...,263.0,0.857143,-0.577350,0.577350,0.561596,-0.839534,0.839534,0.529308,-0.636568,0.636568
6,180599,718059920180202,2018-02-02,B,3,450,F,1957-07-29,Bangor,WI,...,232.0,1.421053,1.059626,1.059626,0.875851,-0.237906,0.237906,1.313621,0.396596,0.396596
7,180599,818059920180326,2018-03-26,B,2,300,F,1957-07-29,Bangor,WI,...,215.0,0.960000,-0.121268,0.121268,0.582314,-0.794031,0.794031,0.900524,-0.127215,0.127215
8,180599,918059920180508,2018-05-08,C,2,150,F,1957-07-29,Bangor,WI,...,201.0,0.545455,-0.944911,0.944911,0.578413,-0.782632,0.782632,0.421678,-0.734522,0.734522
9,180599,1018059920180802,2018-08-02,A,2,200,F,1957-07-29,Bangor,WI,...,184.0,0.923077,-0.218218,0.218218,0.566646,-0.830675,0.830675,0.556522,-0.590305,0.590305


## Lags

Up until now, aggregations over time have been computed. With time-series like data, simple lagged variables can be very powerful.  If only lags of the transaction details were desired, these could have been calculated at any time. However, lagging these aggregations can also be powerful.

Start by grouping by customer as we only want to lag within the customer transactions.

In [22]:
custgroup = transdf.groupby('CustomerID')

Create the lag for the purchase price and date

In [23]:
transdf['Cost_lag_1'] = custgroup['Cost'].shift(1)
transdf['Date_lag_1'] = custgroup['Transaction_Date'].shift(1)

Now we can calculate the difference between the two purchases. In this case, this is the number of days since last purchase.

In [24]:
transdf['Date_Delta_1'] = (transdf['Transaction_Date'] - transdf['Date_lag_1']).dt.days

As mentioned, the lag of more complicated features can be calculated too. In this case, lag the distance from the purchase price to the average.

In [25]:
transdf['Cost_distance_lag_1'] = custgroup['Cost_distance'].shift(1)

Calculate the difference between the current purchase amount and the prior amount. This is useful by itself as a feature.

In [26]:
transdf['Cost_delta_1'] = transdf['Cost'] - transdf['Cost_lag_1']

But when combined with the difference in transaction date can calculate a velocity capturing how the purchase price is changing over time.

In [27]:
transdf['Cost_velocity_days'] = transdf['Cost_delta_1'] / transdf['Date_Delta_1']

Again, computed features can be lagged, like this velocity we just calculated.

In [28]:
transdf['Cost_velocity_days_lag_1'] = custgroup['Cost_velocity_days'].shift(1)
transdf['Cost_velocity_days_delta_1'] = transdf['Cost_velocity_days'] - transdf['Cost_velocity_days_lag_1']

And if the ratio between the change in velocity over the change in time is calculated, this creates a measure of how the purchase amount is accelerating.

In [29]:
transdf['Cost_acceleration_days'] = transdf['Cost_velocity_days_delta_1'] / transdf['Date_Delta_1']

The data from two transactions ago can also be captured as lags.

In [30]:
transdf['Cost_lag_2'] = custgroup['Cost'].shift(2)
transdf['Date_lag_2'] = custgroup['Transaction_Date'].shift(2)
transdf['Date_Delta_2'] = (transdf['Transaction_Date'] - transdf['Date_lag_2']).dt.days
transdf['Cost_distance_lag_2'] = custgroup['Cost_distance'].shift(2)
transdf['Cost_delta_2'] = transdf['Cost'] - transdf['Cost_lag_2']

And the rate of change between the current transaction and the one two transactions ago can be calculated as another velocity. Beyond calculating this velocity over time, it can be calculated over transactions.

In [31]:
transdf['Cost_velocity_days_2'] = transdf['Cost_delta_2'] / transdf['Date_Delta_2']
transdf['Cost_velocity_transactions_2'] = transdf['Cost_delta_2'] / 2

Additional purchase amount lags can be useful.

In [32]:
transdf['Cost_lag_3'] = custgroup['Cost'].shift(3)
transdf['Cost_lag_4'] = custgroup['Cost'].shift(4)

And a weighted average (in this case of the last 5 transactions) can be calculated

In [33]:
transdf['Cost_moving_average_4'] = (5*transdf['Cost'] + 4*transdf['Cost_lag_1'] + 3*transdf['Cost_lag_2'] + 2*transdf['Cost_lag_3'] + transdf['Cost_lag_4'])/15

## Categorical trend variables

We can capture trends as a string, taking numerical variables and creating a categorical one from the sequence. First, we define a function.

In [34]:
def trend_string(row):
    """Use the lag variables from cost to create a string representing the trends. 
'=' - value remains the same, 
'+' - value increasing, 
'-' - value decreasing.
"""
    
    trend = ''
    if np.isnan(row['Cost_lag_1']):
        trend = ' '
    elif row['Cost'] > row['Cost_lag_1']:
        trend = '+'
    elif row['Cost'] < row['Cost_lag_1']:
        trend = '-'
    else:
        trend = '='
    
    if np.isnan(row['Cost_lag_2']):
        trend += ' '
    elif row['Cost_lag_1'] > row['Cost_lag_2']:
        trend += '+'
    elif row['Cost_lag_1'] < row['Cost_lag_2']:
        trend += '-'
    else:
        trend += '='
    
    if np.isnan(row['Cost_lag_3']):
        trend += ' '
    elif row['Cost_lag_2'] > row['Cost_lag_3']:
        trend += '+'
    elif row['Cost_lag_2'] < row['Cost_lag_3']:
        trend += '-'
    else:
        trend += '='
        
    if np.isnan(row['Cost_lag_4']):
        trend += ' '
    elif row['Cost_lag_3'] > row['Cost_lag_4']:
        trend += '+'
    elif row['Cost_lag_3'] < row['Cost_lag_4']:
        trend += '-'
    else:
        trend += '='
    
    return trend

This can then be applied to each transaction.

In [35]:
transdf['Cost_trend'] = transdf.apply(trend_string, axis=1)

transdf[['Cost', 'Cost_lag_1', 'Cost_lag_2', 'Cost_lag_3', 'Cost_lag_4', 'Cost_trend']].head(20)

,Cost,Cost_lag_1,Cost_lag_2,Cost_lag_3,Cost_lag_4,Cost_trend
0,300,NaN,NaN,NaN,NaN,
1,225,300.0,NaN,NaN,NaN,-
2,150,225.0,300.0,NaN,NaN,--
3,200,150.0,225.0,300.0,NaN,+--
4,300,200.0,150.0,225.0,300.0,++--
5,200,300.0,200.0,150.0,225.0,-++-
6,450,200.0,300.0,200.0,150.0,+-++
7,300,450.0,200.0,300.0,200.0,-+-+
8,150,300.0,450.0,200.0,300.0,--+-
9,200,150.0,300.0,450.0,200.0,+--+


### Encoding Categorical Variables

There are multiple ways to encode categorical variables. Before we explore these techniques, we need to setup our target and train-test split.

### Create target

When modeling churn, there are multiple ways to define it. For simplicity, we will simpy tag any transaction within 30 days of final transaction (*Churn_date*) as churn

In [36]:
transdf['Days_to_Churn'] = (transdf['Churn_date'] - transdf['Transaction_Date']).dt.days
transdf['Churn'] = transdf['Days_to_Churn'].apply(lambda x: 1 if not pd.isnull(x) and x <= 30 else 0)
tmpdf = transdf[transdf.Churn_date.apply(lambda x: not pd.isnull(x))]
tmpdf[['Transaction_Date', 'Churn_date', 'Churn']].tail()

,Transaction_Date,Churn_date,Churn
45662,2018-03-11,2018-04-03,1
45663,2018-03-22,2018-04-03,1
45664,2018-03-25,2018-04-03,1
45665,2018-03-31,2018-04-03,1
45666,2018-04-03,2018-04-03,1


### Train-test split

Because of the transactions over time, a random or stratified train-test split will not work. Instead, the data needs to be split on a date with all data before that date in training and all data after that date in test. 

We can calculate the date to use the first 80% of the data for training and the last 20% for testing.

First, calculate the date to split on

In [37]:
min_date = transdf.Transaction_Date.min()
max_date = transdf.Transaction_Date.max()
print("Min:", min_date, "Max:", max_date)

Min: 2017-01-02 00:00:00 Max: 2020-01-01 00:00:00


Calculate the train-test cutoff date at 80% of the way through the data

In [38]:
train_percent = .8
time_between = max_date - min_date
train_cutoff = min_date + train_percent*time_between
train_cutoff

Timestamp('2019-05-27 04:48:00')

Create the train and test dataframes

In [39]:
train_df = transdf[transdf.Transaction_Date <= train_cutoff]
test_df = transdf[transdf.Transaction_Date > train_cutoff]
print("Train:", train_df.Transaction_Date.min(), train_df.Transaction_Date.max())
print("Test:", test_df.Transaction_Date.min(), test_df.Transaction_Date.max())

Train: 2017-01-02 00:00:00 2019-05-27 00:00:00
Test: 2019-05-28 00:00:00 2020-01-01 00:00:00


### One-hot encoding

In [40]:
train_df.select_dtypes('object').columns

Index(['Product', 'Gender', 'City', 'State', 'Customer_since', 'Country',
       'Phone', 'Cost_trend'],
      dtype='object')

We can use the `OneHotEncoder` from **scikit-learn** to create dummy variables for each level of a categorical.  This will work fine for features such as *Product* or *Gender*, but for features such as *City*, *State*, and *Cost_trend* the number of dummy variables needed will be far too large. Instead, in the next section a different encoding method will be discussed.

First, grab just the features of interest.

In [41]:
train_df_ohe = train_df[['Product', 'Gender']].copy()
test_df_ohe = test_df[['Product', 'Gender']].copy()
train_df_ohe.shape

(39236, 2)

Create the encoder and encode the training data.

In [42]:
oh_encoder = OneHotEncoder(handle_unknown="ignore")
train_df_ohe_encoded = oh_encoder.fit_transform(train_df_ohe)
train_df_ohe_encoded.shape

(39236, 7)

Next, apply this encoder to the test set.

In [43]:
test_df_ohe_encode = oh_encoder.transform(test_df_ohe)
test_df_ohe_encode.shape

(6624, 7)

### Target encoding

For high cardinality categoricals, such as *City*, *State*, and *Cost_trend*, one-hot encoding is not ideal. Instead, the categorical value can be encoded by replacing the level with the average target value through the use of `TargetEncoder` from the [category_encoders](https://contrib.scikit-learn.org/category_encoders/) package. This approach bothers some data scientists as it uses each rows target value in the calculation of a feature used to predict it.

Performing the same encoding of the average value of the target for each level, but excluding the current row, mitigates this issue. This approach is know and leave-one-out encoding and is performed by `LeaveOneOutEncoder` in the same package. This notebook will use the leave-one-out approach, but a simple change in the definition of the encoder below will change to target encoding.

Before creating the encoder, both the features to be encoded and a dataframe containing the target need to be created.

In [44]:
train_target = train_df['Churn']

train_df_loo = train_df[['City', 'State', 'Cost_trend']].copy()
test_df_loo = test_df[['City', 'State', 'Cost_trend']].copy()
train_df_loo.shape

(39236, 3)

Create the encoder and fit the training data.

In [45]:
loo_encoder = ce.LeaveOneOutEncoder(return_df=True)
train_df_loo_encode = loo_encoder.fit_transform(train_df_loo, train_target)
train_df_loo_encode.shape

(39236, 3)

Encode the test set.

In [46]:
test_df_loo_encode = loo_encoder.transform(test_df_loo)
test_df_loo_encode.shape

(6624, 3)

In [47]:
train_df_loo_encode.head()

,City,State,Cost_trend
0,0.0,0.007913,0.002002
1,0.0,0.007913,0.002315
2,0.0,0.007913,0.009804
3,0.0,0.007913,0.013158
4,0.0,0.007913,0.007955


The original train and test sets along with the one-hot encoded and leave-one-out encoded categoricals can be merged back together and used in any modeling algorithm.